## Table of Contents
- [Section 1](#section-1)
- [Section 2](#section-2)

<a id="section-1"></a>
## Section 1

Content of section 1

<a id="section-2"></a>
## Section 2

Content of section 2


In [1]:
import os
import pandas as pd
import pickle
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
import time
import shap
import random
from collections import defaultdict
from itertools import combinations
from sklearn.preprocessing import normalize
import warnings
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

# Load Data from DB

- [Load Data fro DB](#Load-Data-from-DB)
- [Start Recommendation Process](#Start-Recommendation-Process)
- [Approach 6](#Approach-6)

In [2]:
def db_connection(query):
    # Database connection parameters
    db_name = 'factihealth'   # Database name
    db_user = 'fh_user'  # Username
    db_password = 'Facti@874'  # Password
    db_host = 'redshift-cluster-factihealth.cuzgotkwtow6.ap-south-1.redshift.amazonaws.com'  # Cluster endpoint
    db_port = 5439  # Port
    # Connect to the database
    try:
        conn = psycopg2.connect(
            dbname=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port
        )
        print("Connected to the database successfully")
        # Create a cursor object
        cur = conn.cursor()
        # Execute a query
        cur.execute(query)
        rows = cur.fetchall()
        
        # Get column names from the cursor description
        col_names = [desc[0] for desc in cur.description]
        
        # Create a DataFrame from the fetched rows and column names
        df = pd.DataFrame(rows, columns=col_names)
        # Close the cursor and connection
        cur.close()
        conn.close()
        
        return df
    except Exception as e:
        print(f"Database connection failed due to {e}")
        
# Get Table Data based on table name
def get_db_data(table_name, patients_ids=()):
    ''' Get Data based on the table name provided '''
    # query = f"""SELECT * FROM factihealth.mimic.{table_name} 
    #             WHERE subject_id in {patients_ids}
    #             """
    query = f"SELECT * FROM factihealth.mimic.{table_name}"
    if patients_ids:
        query += f" WHERE subject_id IN {patients_ids}"
    df = db_connection(query)
    return df

In [3]:
admissions_df = get_db_data('admissions')
display(admissions_df.head())
admissions_df.shape

Connected to the database successfully


,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,12024697,20302177,2109-12-14 22:50:00,2110-01-15 14:53:00,NaT,EW EMER.,P38V7W,EMERGENCY ROOM,REHAB,Other,ENGLISH,MARRIED,WHITE,2109-12-14 19:31:00,2109-12-15 01:56:00,0
1,15350437,20383396,2110-01-11 08:02:00,2110-01-12 18:45:00,NaT,EU OBSERVATION,P65DYX,TRANSFER FROM HOSPITAL,,Other,ENGLISH,SINGLE,WHITE,2110-01-11 03:43:00,2110-01-11 08:41:00,0
2,18106347,24305596,2110-01-11 10:14:00,2110-01-15 17:31:00,NaT,EW EMER.,P41R5N,PACU,HOME HEALTH CARE,Other,ENGLISH,DIVORCED,WHITE,NaT,NaT,0
3,17195991,23542772,2110-01-11 22:47:00,2110-01-18 10:25:00,NaT,EW EMER.,P58A9J,EMERGENCY ROOM,CHRONIC/LONG TERM ACUTE CARE,Other,ENGLISH,SINGLE,UNABLE TO OBTAIN,2110-01-11 21:42:00,2110-01-12 00:54:00,0
4,11274131,28390257,2110-01-12 22:11:00,2110-01-14 15:31:00,NaT,EW EMER.,P99698,EMERGENCY ROOM,HOME,Other,ENGLISH,MARRIED,WHITE,2110-01-12 18:52:00,2110-01-12 23:31:00,0


(862462, 16)

In [4]:
patient_id_list = sorted(admissions_df.subject_id.unique())
patient_id_list = tuple(random.sample(patient_id_list, 1000))
#patient_id_list

In [5]:
patients_df = get_db_data('patients', patient_id_list)
display(patients_df.head())
patients_df.shape

Connected to the database successfully


,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10011231,F,42,2146,2008 - 2010,None
1,10023708,F,82,2143,2008 - 2010,None
2,10036642,M,26,2159,2014 - 2016,None
3,10040884,M,83,2162,2011 - 2013,2162-09-01
4,10060863,F,59,2192,2011 - 2013,None


(1000, 6)

In [6]:
admissions_df = get_db_data('admissions', patient_id_list)
display(admissions_df.head())
admissions_df.shape

Connected to the database successfully


,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,11387700,24577944,2110-07-23 07:15:00,2110-08-12 13:30:00,NaT,SURGICAL SAME DAY ADMISSION,P41R5N,PHYSICIAN REFERRAL,REHAB,Medicare,ENGLISH,MARRIED,WHITE,NaT,NaT,0
1,16874539,26844235,2110-08-04 20:49:00,2110-08-08 16:00:00,NaT,URGENT,P4102Q,PHYSICIAN REFERRAL,HOME,Other,ENGLISH,MARRIED,UNKNOWN,2110-08-04 19:57:00,2110-08-04 19:57:00,0
2,17886844,20008372,2110-10-30 08:03:00,2110-11-04 17:05:00,NaT,EW EMER.,P00P3O,EMERGENCY ROOM,SKILLED NURSING FACILITY,Other,?,,UNKNOWN,2110-10-30 04:38:00,2110-10-30 09:48:00,0
3,12112501,21599835,2110-11-23 07:15:00,2110-11-25 20:10:00,NaT,SURGICAL SAME DAY ADMISSION,P2720E,PHYSICIAN REFERRAL,HOME,Other,ENGLISH,MARRIED,WHITE,NaT,NaT,0
4,14172190,27215730,2111-01-28 22:32:00,2111-01-29 13:20:00,2111-01-29 13:20:00,EW EMER.,P38TI6,EMERGENCY ROOM,DIED,Other,ENGLISH,,OTHER,2111-01-28 21:02:00,2111-01-29 00:39:00,1


(4440, 16)

In [7]:
admissions_df.subject_id.nunique()

1000

In [8]:
prescriptions_df = get_db_data('prescriptions', patient_id_list)
display(prescriptions_df.head())
prescriptions_df.shape

Connected to the database successfully


,subject_id,hadm_id,pharmacy_id,poe_id,poe_seq,order_provider_id,starttime,stoptime,drug_type,drug,formulary_drug_cd,gsn,ndc,prod_strength,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route
0,10020944,29974575,5323857,10020944-288,288.0,P96HOQ,2131-02-28 01:00:00,2131-03-13 21:00:00,MAIN,ClonazePAM,CLON5,004560,5.107909e+10,0.5mg Tablet,,0.25,mg,0.5,TAB,2,PO/NG
1,10020944,29974575,8353553,10020944-294,294.0,,2131-03-02 16:00:00,2131-03-09 18:00:00,MAIN,Sodium Chloride 0.9% Flush,NACLFLUSH,,0.000000e+00,10 mL Syringe,,10-20,mL,1-2,SYR,1,IV
2,10020944,29974575,11192142,10020944-79,79.0,P96HOQ,2131-02-27 20:00:00,2131-03-07 20:00:00,MAIN,Chlorhexidine Gluconate 0.12% Oral Rinse,CHLO15L,057959,5.456952e+10,15ml Cup,,15,mL,1,UDCUP,2,ORAL
3,10020944,29974575,14360659,10020944-295,295.0,P743BY,2131-03-06 15:00:00,2131-03-09 16:00:00,BASE,Iso-Osmotic Dextrose,FRBD100,,0.000000e+00,100ml Bag,,100,mL,100,mL,3,IV
4,10020944,29974575,14706535,10020944-296,296.0,P743BY,2131-03-08 11:00:00,2131-03-13 21:00:00,MAIN,Vitamin D,VITD1000,028465,9.045825e+08,"1,000 Unit Tablet",,2000,UNIT,2,TAB,2,PO/NG


(78420, 21)

In [9]:
prescriptions_df.subject_id.nunique()

879

In [10]:
diagnosis_df = get_db_data('diagnosis_icd', patient_id_list)
display(diagnosis_df.head())
diagnosis_df.shape

Connected to the database successfully


,subject_id,hadm_id,seq_num,icd_code,icd_version
0,12159880,23239637,2,00845,9
1,12159880,23518380,3,00845,9
2,12426769,26860378,5,00845,9
3,14146380,27557095,1,00845,9
4,15204989,20274215,1,00845,9


(24419, 5)

In [11]:
diagnosis_df.subject_id.nunique()

999

In [12]:
chartevents_df = get_db_data('chartevents', patient_id_list)
display(chartevents_df.head())
chartevents_df.shape

Connected to the database successfully


,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,10020944,29974575,30757476,6887.0,2131-03-01 08:00:00,2131-03-01 08:27:00,220277,100,100.0,%,0.0
1,10020944,29974575,30757476,6887.0,2131-03-01 08:00:00,2131-03-01 08:27:00,220293,19,19.0,L/min,0.0
2,10020944,29974575,30757476,6887.0,2131-03-01 08:00:00,2131-03-01 08:27:00,220339,8,8.0,cmH2O,0.0
3,10020944,29974575,30757476,6887.0,2131-03-01 08:00:00,2131-03-01 08:27:00,223835,40,40.0,,0.0
4,10020944,29974575,30757476,6887.0,2131-03-01 08:00:00,2131-03-01 08:27:00,223848,Drager,1.0,,0.0


(1849457, 11)

In [13]:
chartevents_df.subject_id.nunique()

268

In [14]:

def categorize_icd(icd):
    try:
        if 1 <= int(icd) <= 139:
            return 'Infectious and Parasitic Diseases'
        elif 140 <= int(icd) <= 239:
            return 'Neoplasms'
        elif 240 <= int(icd) <= 279:
            return 'Endocrine, Nutritional, and Metabolic Diseases'
        elif 280 <= int(icd) <= 289:
            return 'Diseases of the Blood and Blood-forming Organs'
        elif 290 <= int(icd) <= 319:
            return 'Mental Disorders'
        elif 320 <= int(icd) <= 389:
            return 'Diseases of the Nervous System and Sense Organs'
        elif 390 <= int(icd) <= 459:
            return 'Diseases of the Circulatory System'
        elif 460 <= int(icd) <= 519:
            return 'Diseases of the Respiratory System'
        elif 520 <= int(icd) <= 579:
            return 'Diseases of the Digestive System'
        elif 580 <= int(icd) <= 629:
            return 'Diseases of the Genitourinary System'
        elif 630 <= int(icd) <= 679:
            return 'Complications of Pregnancy, Childbirth, and the Puerperium'
        elif 680 <= int(icd) <= 709:
            return 'Diseases of the Skin and Subcutaneous Tissue'
        elif 710 <= int(icd) <= 739:
            return 'Diseases of the Musculoskeletal System and Connective Tissue'
        elif 740 <= int(icd) <= 759:
            return 'Congenital Anomalies'
        elif 760 <= int(icd) <= 779:
            return 'Certain Conditions Originating in the Perinatal Period'
        elif 780 <= int(icd) <= 799:
            return 'Symptoms, Signs, and Ill-defined Conditions'
        elif 800 <= int(icd) <= 999:
            return 'Injury and Poisoning'
        
    except ValueError:
        if icd[0] == 'A' or icd[0] == 'B':
            return 'Infectious and Parasitic Diseases'
        elif icd[0] == 'C':
            return 'Neoplasms'
        elif icd[0] == 'D':
            return 'Diseases of the Blood and Blood-forming Organs'
        elif icd[0] == 'E':
            return 'External causes of injury and supplemental classification'
        elif icd[0] == 'F':
            return 'Mental Disorders'
        elif icd[0] == 'G':
            return 'Diseases of the Nervous System and Sense Organs'
        elif icd[0] == 'H':
            return 'Diseases of the eye, adnexa and mastoid process'
        elif icd[0] == 'I':
            return 'Diseases of the Circulatory System'
        elif icd[0] == 'J':
            return 'Diseases of the Respiratory System'
        elif icd[0] == 'K':
            return 'Diseases of the Digestive System'
        elif icd[0] == 'L':
            return 'Diseases of the Skin and Subcutaneous Tissue'
        elif icd[0] == 'M':
            return 'Diseases of the Musculoskeletal System and Connective Tissue'
        elif icd[0] == 'N':
            return 'Diseases of the Genitourinary System'
        elif icd[0] == 'O':
            return 'Complications of Pregnancy, Childbirth, and the Puerperium'
        elif icd[0] == 'P':
            return 'Certain Conditions Originating in the Perinatal Period'
        elif icd[0] == 'Q':
            return 'Congenital Anomalies'
        elif icd[0] == 'R':
            return 'Symptoms, Signs, and Ill-defined Conditions'
        elif icd[0] == 'S' or icd[0] == 'T' :
            return 'Injury and Poisoning'
        elif icd[0] == 'U':
            return 'Codes for special purposes'
        elif icd[0] in ['V', 'W', 'X' ,'Y']:
            return 'External causes of injury and supplemental classification'
        elif icd[0] == 'Z':
            return 'Factors influencing health status and contact with health services'

        
## Categorize Diagnosis Data with the predefined ICD Codes
diagnosis_df['icd'] = diagnosis_df.icd_code.str[:3]
diagnosis_df['diagnosis'] = diagnosis_df['icd'].apply(categorize_icd)
diagnosis_df = diagnosis_df[['subject_id', 'hadm_id', 'diagnosis']]
diagnosis_df.head()

,subject_id,hadm_id,diagnosis
0,12159880,23239637,Infectious and Parasitic Diseases
1,12159880,23518380,Infectious and Parasitic Diseases
2,12426769,26860378,Infectious and Parasitic Diseases
3,14146380,27557095,Infectious and Parasitic Diseases
4,15204989,20274215,Infectious and Parasitic Diseases


In [15]:
## Merge Diagnosis Data with Diagnosis Code(ICD)
patients_df = patients_df[['subject_id', 'gender', 'anchor_age']]
pat_diagn_data = pd.merge(diagnosis_df, patients_df, on='subject_id', how='left')
pat_diagn_data.drop_duplicates(inplace=True)
print(pat_diagn_data.shape)
pat_diagn_data.head()

(12557, 5)


,subject_id,hadm_id,diagnosis,gender,anchor_age
0,12159880,23239637,Infectious and Parasitic Diseases,F,80
1,12159880,23518380,Infectious and Parasitic Diseases,F,80
2,12426769,26860378,Infectious and Parasitic Diseases,F,33
3,14146380,27557095,Infectious and Parasitic Diseases,F,74
4,15204989,20274215,Infectious and Parasitic Diseases,F,80


In [16]:
## MERGE WITH THE ADMISSION DATA
admin_diag = pd.merge(admissions_df, pat_diagn_data, on=['hadm_id', 'subject_id'], 
                          how='left')
print(admin_diag.shape)
admin_diag.head()

(25118, 19)


,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,diagnosis,gender,anchor_age
0,11387700,24577944,2110-07-23 07:15:00,2110-08-12 13:30:00,NaT,SURGICAL SAME DAY ADMISSION,P41R5N,PHYSICIAN REFERRAL,REHAB,Medicare,ENGLISH,MARRIED,WHITE,NaT,NaT,0,"Endocrine, Nutritional, and Metabolic Diseases",M,87.0
1,11387700,24577944,2110-07-23 07:15:00,2110-08-12 13:30:00,NaT,SURGICAL SAME DAY ADMISSION,P41R5N,PHYSICIAN REFERRAL,REHAB,Medicare,ENGLISH,MARRIED,WHITE,NaT,NaT,0,Diseases of the Circulatory System,M,87.0
2,11387700,24577944,2110-07-23 07:15:00,2110-08-12 13:30:00,NaT,SURGICAL SAME DAY ADMISSION,P41R5N,PHYSICIAN REFERRAL,REHAB,Medicare,ENGLISH,MARRIED,WHITE,NaT,NaT,0,Diseases of the Genitourinary System,M,87.0
3,11387700,24577944,2110-07-23 07:15:00,2110-08-12 13:30:00,NaT,SURGICAL SAME DAY ADMISSION,P41R5N,PHYSICIAN REFERRAL,REHAB,Medicare,ENGLISH,MARRIED,WHITE,NaT,NaT,0,Diseases of the Musculoskeletal System and Con...,M,87.0
4,11387700,24577944,2110-07-23 07:15:00,2110-08-12 13:30:00,NaT,SURGICAL SAME DAY ADMISSION,P41R5N,PHYSICIAN REFERRAL,REHAB,Medicare,ENGLISH,MARRIED,WHITE,NaT,NaT,0,"Symptoms, Signs, and Ill-defined Conditions",M,87.0


In [17]:
## Select the required Columns
admin_diag = admin_diag[['subject_id', 'hadm_id', 'admittime','admission_type',
                         'admission_location','discharge_location', 'insurance', 
                         'language', 'marital_status', 'race',
                         'gender', 'anchor_age', 'diagnosis']]
admin_diag.head(2)

,subject_id,hadm_id,admittime,admission_type,admission_location,discharge_location,insurance,language,marital_status,race,gender,anchor_age,diagnosis
0,11387700,24577944,2110-07-23 07:15:00,SURGICAL SAME DAY ADMISSION,PHYSICIAN REFERRAL,REHAB,Medicare,ENGLISH,MARRIED,WHITE,M,87.0,"Endocrine, Nutritional, and Metabolic Diseases"
1,11387700,24577944,2110-07-23 07:15:00,SURGICAL SAME DAY ADMISSION,PHYSICIAN REFERRAL,REHAB,Medicare,ENGLISH,MARRIED,WHITE,M,87.0,Diseases of the Circulatory System


In [18]:
patient_data = pd.merge(admin_diag, prescriptions_df, on=['hadm_id', 'subject_id'], how='left')
print(patient_data.shape)
patient_data['admit_date'] = pd.to_datetime(patient_data['admittime']).dt.date
patient_data.head(3)

(1191290, 32)


,subject_id,hadm_id,admittime,admission_type,admission_location,discharge_location,insurance,language,marital_status,race,gender,anchor_age,diagnosis,pharmacy_id,poe_id,poe_seq,order_provider_id,starttime,stoptime,drug_type,drug,formulary_drug_cd,gsn,ndc,prod_strength,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route,admit_date
0,11387700,24577944,2110-07-23 07:15:00,SURGICAL SAME DAY ADMISSION,PHYSICIAN REFERRAL,REHAB,Medicare,ENGLISH,MARRIED,WHITE,M,87.0,"Endocrine, Nutritional, and Metabolic Diseases",35042.0,11387700-2,2.0,P38GCF,2110-07-23 14:00:00,2110-08-12 18:00:00,MAIN,Influenza Virus Vaccine,INFV,064182,5.816009e+10,0.5mL Syringe,,0.5,mL,1,SYR,,IM,2110-07-23
1,11387700,24577944,2110-07-23 07:15:00,SURGICAL SAME DAY ADMISSION,PHYSICIAN REFERRAL,REHAB,Medicare,ENGLISH,MARRIED,WHITE,M,87.0,"Endocrine, Nutritional, and Metabolic Diseases",348940.0,11387700-343,343.0,P18Z04,2110-08-05 20:00:00,2110-08-06 06:00:00,BASE,D5NS,D5NS1000,002001,3.380089e+08,1000mL Bag,,1000,mL,1000,mL,,IV,2110-07-23
2,11387700,24577944,2110-07-23 07:15:00,SURGICAL SAME DAY ADMISSION,PHYSICIAN REFERRAL,REHAB,Medicare,ENGLISH,MARRIED,WHITE,M,87.0,"Endocrine, Nutritional, and Metabolic Diseases",3009660.0,11387700-307,307.0,P451D0,2110-08-03 11:00:00,2110-08-06 06:00:00,BASE,Vial,VIAL,,0.000000e+00,Send Vial,,1,VIAL,1,VIAL,1,IV,2110-07-23


In [19]:
patient_data.subject_id.nunique()

1000

In [20]:

def categorize_race(admin_diag):
    ## Recategorize Race
    admin_diag['race'] = np.where(admin_diag['race'].isin(['AMERICAN INDIAN/ALASKA NATIVE']), 'AMERICAN INDIAN/ALASKA NATIVE',
                            np.where(admin_diag['race'].isin(['ASIAN', 'ASIAN - ASIAN INDIAN', 'ASIAN - CHINESE',
                                                                           'ASIAN - KOREAN', 'ASIAN - SOUTH EAST ASIAN']),
                                     'ASIAN',
                            np.where(admin_diag['race'].isin(['BLACK/AFRICAN', 'BLACK/AFRICAN AMERICAN', 
                                                                           'BLACK/CAPE VERDEAN', 'BLACK/CARIBBEAN ISLAND']),
                                     'BLACK/AFRICAN',
                            np.where(admin_diag['race'].isin(['HISPANIC OR LATINO', 'HISPANIC/LATINO - CENTRAL AMERICAN',
                                                                           'HISPANIC/LATINO - COLUMBIAN', 'HISPANIC/LATINO - CUBAN',
                                                                           'HISPANIC/LATINO - DOMINICAN', 'HISPANIC/LATINO - GUATEMALAN', 
                                                                           'HISPANIC/LATINO - HONDURAN', 'HISPANIC/LATINO - MEXICAN', 
                                                                           'HISPANIC/LATINO - PUERTO RICAN','HISPANIC/LATINO - SALVADORAN']), 
                                     'HISPANIC OR LATINO',
                            np.where(admin_diag['race'].isin(['NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER']), 
                                     'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER',
                            np.where(admin_diag['race'].isin(['MULTIPLE RACE/race', 'OTHER', 
                                                                      'PATIENT DECLINED TO ANSWER', 'UNABLE TO OBTAIN', 'UNKNOWN']), 
                                     'OTHER',
                            np.where(admin_diag['race'].isin(['PORTUGUESE']), 'PORTUGUESE',
                            np.where(admin_diag['race'].isin(['SOUTH AMERICAN']), 'SOUTH AMERICAN',
                            np.where(admin_diag['race'].isin(['WHITE', 'WHITE - BRAZILIAN', 'WHITE - EASTERN EUROPEAN',
                                                                           'WHITE - OTHER EUROPEAN', 'WHITE - RUSSIAN']), 
                                     'WHITE',
                                     np.nan)))))))))
    

    return admin_diag


# CATEGPROIZE RACES
patient_data = categorize_race(patient_data)
patient_data.shape

(1191290, 33)

In [21]:
## Recategorize Admission Type
conditions = patient_data['admission_type'].isin(['EW EMER.', 'URGENT', 'DIRECT EMER.'])
patient_data['admission_type'] = np.where(conditions, 'Emergency', patient_data['admission_type'])

conditions = patient_data['admission_type'].isin(['OBSERVATION ADMIT', 'EU OBSERVATION', 'DIRECT OBSERVATION', 'AMBULATORY OBSERVATION'])
patient_data['admission_type'] = np.where(conditions, 'Observation', patient_data['admission_type'])

conditions = patient_data['admission_type'].isin(['SURGICAL SAME DAY ADMISSION'])
patient_data['admission_type'] = np.where(conditions, 'Surgical Same Day Admission', patient_data['admission_type'])

conditions = patient_data['admission_type'].isin(['ELECTIVE'])
patient_data['admission_type'] = np.where(conditions, 'Elective', patient_data['admission_type'])


In [22]:
patient_data.head()

,subject_id,hadm_id,admittime,admission_type,admission_location,discharge_location,insurance,language,marital_status,race,gender,anchor_age,diagnosis,pharmacy_id,poe_id,poe_seq,order_provider_id,starttime,stoptime,drug_type,drug,formulary_drug_cd,gsn,ndc,prod_strength,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route,admit_date
0,11387700,24577944,2110-07-23 07:15:00,Surgical Same Day Admission,PHYSICIAN REFERRAL,REHAB,Medicare,ENGLISH,MARRIED,WHITE,M,87.0,"Endocrine, Nutritional, and Metabolic Diseases",35042.0,11387700-2,2.0,P38GCF,2110-07-23 14:00:00,2110-08-12 18:00:00,MAIN,Influenza Virus Vaccine,INFV,064182,5.816009e+10,0.5mL Syringe,,0.5,mL,1,SYR,,IM,2110-07-23
1,11387700,24577944,2110-07-23 07:15:00,Surgical Same Day Admission,PHYSICIAN REFERRAL,REHAB,Medicare,ENGLISH,MARRIED,WHITE,M,87.0,"Endocrine, Nutritional, and Metabolic Diseases",348940.0,11387700-343,343.0,P18Z04,2110-08-05 20:00:00,2110-08-06 06:00:00,BASE,D5NS,D5NS1000,002001,3.380089e+08,1000mL Bag,,1000,mL,1000,mL,,IV,2110-07-23
2,11387700,24577944,2110-07-23 07:15:00,Surgical Same Day Admission,PHYSICIAN REFERRAL,REHAB,Medicare,ENGLISH,MARRIED,WHITE,M,87.0,"Endocrine, Nutritional, and Metabolic Diseases",3009660.0,11387700-307,307.0,P451D0,2110-08-03 11:00:00,2110-08-06 06:00:00,BASE,Vial,VIAL,,0.000000e+00,Send Vial,,1,VIAL,1,VIAL,1,IV,2110-07-23
3,11387700,24577944,2110-07-23 07:15:00,Surgical Same Day Admission,PHYSICIAN REFERRAL,REHAB,Medicare,ENGLISH,MARRIED,WHITE,M,87.0,"Endocrine, Nutritional, and Metabolic Diseases",3368481.0,11387700-376,376.0,P45MAB,2110-08-08 22:00:00,2110-08-09 07:00:00,MAIN,Insulin,GLAR100I,047780,8.822203e+07,10mLVial,,30,UNIT,0.03,VIAL,1,SC,2110-07-23
4,11387700,24577944,2110-07-23 07:15:00,Surgical Same Day Admission,PHYSICIAN REFERRAL,REHAB,Medicare,ENGLISH,MARRIED,WHITE,M,87.0,"Endocrine, Nutritional, and Metabolic Diseases",5199446.0,11387700-35,35.0,P38GCF,2110-07-23 14:00:00,2110-08-12 18:00:00,BASE,SW,KCLBASE2,,0.000000e+00,50ml Bag,,50,mL,50,mL,,IV,2110-07-23


In [23]:
patient_data.form_unit_disp.nunique()

50

In [24]:
patient_data.dropna(inplace=True)

In [25]:
patient_data.shape

(1179374, 33)

# Start Recommendation Process

- [Load Data fro DB](#Load-Data-from-DB)
- [Start Recommendation Process](#Start-Recommendation-Process)
- [Approach 6](#Approach-6)


## Type 1 (with Admit Date)

In [26]:
drug_diag_df = patient_data[['admit_date','diagnosis', 'drug']].sort_values('drug')
drug_diag_df.dropna(inplace=True)
print(drug_diag_df.shape)
drug_diag_df.drop_duplicates(inplace=True)
drug_diag_df.tail()

(1243850, 3)


,admit_date,diagnosis,drug
204406,2136-02-28,Diseases of the Digestive System,vancomycin
203844,2136-02-28,Diseases of the Circulatory System,vancomycin
203563,2136-02-28,Mental Disorders,vancomycin
203282,2136-02-28,External causes of injury and supplemental cla...,vancomycin
203001,2136-02-28,Diseases of the Blood and Blood-forming Organs,vancomycin


In [27]:
# Type 2 (without Admit Date)

In [28]:
print(drug_diag_df.shape)

(330750, 3)


In [29]:
drug_diag_pivot = drug_diag_df.pivot_table(index='drug', columns='diagnosis', aggfunc=lambda x: True).fillna(False)
drug_diag_pivot.tail()

admit_date  \
diagnosis                      Complications of Pregnancy, Childbirth, and the Puerperium   
drug                                                                                        
tolcapone                                                                   False           
traZODONE                                                                   False           
urea                                                                        False           
valproic acid (as sodium salt)                                              False           
vancomycin                                                                  False           

                                                     \
diagnosis                      Congenital Anomalies   
drug                                                  
tolcapone                                     False   
traZODONE                                      True   
urea                                          False   
valproic acid (as sodium salt)                False   
vancomycin                                    False   

                                                                               \
diagnosis                      Diseases of the Blood and Blood-forming Organs   
drug                                                                            
tolcapone                                                               False   
traZODONE                                                                True   
urea                                                                    False   
valproic acid (as sodium salt)                                           True   
vancomycin                                                               True   

                                                                   \
diagnosis                      Diseases of the Circulatory System   
drug                                                                
tolcapone                                                    True   
traZODONE                                                    True   
urea                                                         True   
valproic acid (as sodium salt)                               True   
vancomycin                                                   True   

                                                                 \
diagnosis                      Diseases of the Digestive System   
drug                                                              
tolcapone                                                  True   
traZODONE                                                  True   
urea                                                       True   
valproic acid (as sodium salt)                             True   
vancomycin                                                 True   

                                                                     \
diagnosis                      Diseases of the Genitourinary System   
drug                                                                  
tolcapone                                                      True   
traZODONE                                                      True   
urea                                                          False   
valproic acid (as sodium salt)                                 True   
vancomycin                                                     True   

                                                                                             \
diagnosis                      Diseases of the Musculoskeletal System and Connective Tissue   
drug                                                                                          
tolcapone                                                                    True             
traZODONE                                                                    True             
urea                                                                        False             
valproic acid (

In [30]:
drug_diag_pivot.corrwith(drug_diag_pivot.loc[:, ('admit_date', 'Neoplasms')]).sort_values(ascending=False)


            diagnosis                                                         
admit_date  Neoplasms                                                             1.000000
            Congenital Anomalies                                                  0.405725
            Infectious and Parasitic Diseases                                     0.400777
            Diseases of the Skin and Subcutaneous Tissue                          0.395774
            Diseases of the Musculoskeletal System and Connective Tissue          0.389409
            Diseases of the eye, adnexa and mastoid process                       0.376422
            Injury and Poisoning                                                  0.370151
            Diseases of the Genitourinary System                                  0.341268
            Symptoms, Signs, and Ill-defined Conditions                           0.339083
            Diseases of the Respiratory System                                    0.333429
           

In [31]:
# List to store correlation results for each date
correlation_results = []

# Iterate over unique dates
for date in drug_diag_df['admit_date'].unique():
    # Filter the pivot table for the current date
    pivot_for_date = drug_diag_pivot.loc[drug_diag_pivot.index.isin(drug_diag_df[drug_diag_df['admit_date'] == date]['drug'])]
    # Calculate the correlation with 'Neoplasms' for the current date
    correlation_with_neoplasms = pivot_for_date.corrwith(pivot_for_date.loc[:, ('admit_date', 'Neoplasms')]).sort_values(ascending=False)
    # Append the correlation results for the current date to the list
    correlation_results.append((date, correlation_with_neoplasms))

# Print the correlation results
for date, correlations in correlation_results:
    print("Correlation results for date:", date)
    print(correlations)
    print()


KeyboardInterrupt: 

# Type 2 (without Admit Date)

In [ ]:
drug_diag_df = patient_data[['diagnosis', 'drug']].sort_values('drug')
drug_diag_df.dropna(inplace=True)
print(drug_diag_df.shape)
drug_diag_df.drop_duplicates(inplace=True)
drug_diag_df.tail()

In [ ]:
print(drug_diag_df.shape)

In [ ]:
drug_diag_pivot = drug_diag_df.pivot_table(index='drug', columns='diagnosis', aggfunc=lambda x: True).fillna(False)
drug_diag_pivot.tail()

In [ ]:
drug_diag_pivot.corrwith(drug_diag_pivot.loc[:, ('Neoplasms')]).sort_values(ascending=False)


In [ ]:
## Not on Admit Date

drug_diag_df = patient_data[['diagnosis', 'drug']].sort_values('drug')
drug_diag_df.dropna(inplace=True)

drug_diag_pivot = drug_diag_df.pivot_table(index='drug', columns='diagnosis', aggfunc=lambda x: True).fillna(False)

drug_diag_pivot
drug_diag_pivot.corrwith(drug_diag_pivot['Neoplasms']).sort_values(ascending=False)

In [ ]:
patient_data.head(1)

In [ ]:
patient_data.diagnosis.unique()

In [ ]:
for diagnosis in patient_data.diagnosis.unique():
    drug_diag_pivot.corrwith(drug_diag_pivot[diagnosis]).sort_values(ascending=False).head().reset_index()[1:]['diagnosis'].tolist()

In [ ]:
    drug_diag_pivot.corrwith(drug_diag_pivot[diagnosis]).sort_values(ascending=False).head().reset_index()[1:]['diagnosis'].tolist()

In [ ]:
patient_data.groupby(['subject_id', 'hadm_id', 'drug'])['diagnosis'].count()

In [ ]:
df1 = patient_data[patient_data.hadm_id==25491636]
df1

In [ ]:
len(set(df1.groupby(['diagnosis'])['drug'].unique().reset_index().loc[0]['drug']))

In [ ]:
df1.groupby(['diagnosis'])['drug'].unique()

In [ ]:
df = df1.groupby(['diagnosis'])['drug'].unique().reset_index()
df

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import pandas as pd

In [ ]:
# Sample dataset representing diagnoses and associated drugs
data = {'diagnosis': ['Dg1', 'Dg2', 'Dg1', 'Dg2', 'Dg1'],
        'drug': [['Drug1', 'Drug2', 'Drug3', 'Drug4'],
                  ['Drug1', 'Drug5', 'Drug6', 'Drug7', 'Drug9'],
                  ['Drug1', 'Drug12', 'Drug31', 'Drug4'],
                  ['Drug10', 'Drug5', 'Drug16', 'Drug17', 'Drug9'],
                  ['Drug1', 'Drug2', 'Drug3', 'Drug4']]}
df = pd.DataFrame(data)
df

In [ ]:
drug_diag_df

In [ ]:
# Convert list of drugs to a string for each transaction
df =drug_diag_df.copy(deep=True)
# df['drug'] = df['drug'].apply(lambda x: ','.join(x))
df

In [ ]:
# Apply one-hot encoding to create transaction-item matrix
oht = df.groupby(['diagnosis', 'drug']).size().unstack().reset_index().set_index('diagnosis')
oht = oht.fillna(0).applymap(lambda x: 1 if x > 0 else 0)  # Replace NaN with 0 and convert counts to binary
print(oht.shape)
oht

In [ ]:
# Apply Apriori algorithm to find frequent itemsets
frequent_itemsets = apriori(oht, min_support=0.5, use_colnames=True)
frequent_itemsets

In [ ]:
# Apply association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules

In [ ]:
# Print discovered association rules
print("Discovered Association Rules:")
rules

# KNN - Approach 4

In [ ]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

drug_diag_matrix = csr_matrix(drug_diag_pivot.values)
drug_diag_matrix

In [ ]:
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(drug_diag_matrix)
model_knn.get_params()

In [ ]:
drug_diag_pivot.shape

In [ ]:
# query_index = np.random.choice(drug_diag_pivot.shape[0])
query_index = 17

print(query_index)
distances, indices = model_knn.kneighbors(drug_diag_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

In [ ]:
distances.flatten()

In [ ]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(drug_diag_pivot.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, drug_diag_pivot.index[indices.flatten()[i]], distances.flatten()[i]))


In [ ]:
## Check the patients who have Neoplasm, and is the results close enough

patient_data[patient_data.diagnosis=='Neoplasms']['subject_id'].unique()

In [ ]:
patient_data[patient_data.subject_id == 19484821]['diagnosis'].unique()

In [ ]:
patient_data[patient_data.subject_id == 19763115]['diagnosis'].unique()

In [ ]:
# Filter rows where the diagnosis is Fever
fever_df = patient_data[patient_data.diagnosis=='Neoplasms']

# Get unique patient IDs where Fever was diagnosed
fever_patient_ids = fever_df['subject_id'].unique()

# Initialize an empty list to store pairs of patient IDs and corresponding diagnoses
pairs = []

# Iterate over each patient ID with Fever
for patient_id in fever_patient_ids:
    # Get other diagnoses for this patient
    other_diagnoses = patient_data[(patient_data['subject_id'] == patient_id) &
                                   (patient_data['diagnosis'] != 'Neoplasms')]['diagnosis'].unique()
    # Append pair of patient ID and corresponding other diagnoses to the list
    pairs.append((patient_id, other_diagnoses))

# Print pairs
for pair in pairs:
    print("Patient ID:", pair[0])
    print("Other Diagnoses:", pair[1])
    print()

In [ ]:
## To see the most common diagnosis

import pandas as pd

# Assuming your dataset is stored in a DataFrame called df
# Replace df with your actual DataFrame name
diag= 'Neoplasms'

# Filter rows where the diagnosis is Fever
selected_diagnosis_df = patient_data[patient_data.diagnosis==diag]

# Get other diagnoses for patients with Fever
other_diagnoses = patient_data[(patient_data['subject_id'].isin(selected_diagnosis_df['subject_id'])) & 
                               (patient_data['diagnosis'] != diag)]

# Count occurrences of each diagnosis
diagnosis_counts = other_diagnoses['diagnosis'].value_counts(normalize=True)*100

# Print the most common diagnosis with Fever
most_common_diagnosis = diagnosis_counts[:4]
count = diagnosis_counts.max()
print("Most common diagnosis with Diagnosis:", most_common_diagnosis)
print("Count:", count)


In [ ]:
import pandas as pd

# Assuming your dataset is stored in a DataFrame called patient_data
# Replace patient_data with your actual DataFrame name

# Filter rows where the diagnosis is Neoplasms
neoplasms_df = patient_data[patient_data['diagnosis'] == 'Neoplasms']

# Get subject_ids where Neoplasms were diagnosed
neoplasms_subject_ids = neoplasms_df['subject_id'].unique()

# Filter rows where the diagnosis is not Neoplasms and subject_id is in neoplasms_subject_ids
other_diagnoses = patient_data[(patient_data['subject_id'].isin(neoplasms_subject_ids)) &
                               (patient_data['diagnosis'] != 'Neoplasms')]

# Group by subject_id, admit_date, and diagnosis, then count occurrences of each diagnosis for each group
diagnosis_counts_by_subject_date = other_diagnoses.groupby(['subject_id', 'admit_date', 'diagnosis']).size()

# Reset index to prepare for grouping by subject_id and admit_date only
diagnosis_counts_by_subject_date = diagnosis_counts_by_subject_date.reset_index(name='count')

# Get the most common diagnoses for each subject_id and admit_date
most_common_diagnoses_by_subject_date = diagnosis_counts_by_subject_date.groupby(['subject_id', 'admit_date']).apply(lambda x: x.nlargest(5, 'count'))

# Print the most common diagnoses for each subject_id and admit_date
print("Most common diagnoses for each subject_id and admit_date:")
most_common_diagnoses_by_subject_date#.reset_index()


In [ ]:
other_diagnoses['admit_date'].nunique()

In [ ]:
drug_diag_df[drug_diag_df['drug'].str.contains('Propoxyphene')]

In [ ]:
drug_diag_df[drug_diag_df.diagnosis=='Symptoms, Signs, and Ill-defined Conditions']#['drug'].nunique()

In [ ]:
import datetime

In [ ]:
# df=patient_data[(patient_data.diagnosis=='Symptoms, Signs, and Ill-defined Conditions') & 
#              (patient_data.subject_id==14878942) &
#              (patient_data.admit_date==datetime.date(2110, 9, 7))]
# df

df=patient_data[(patient_data.subject_id==14878942) & 
#              (patient_data.subject_id==14878942) &
             (patient_data.admit_date==datetime.date(2110, 9, 7))]
df

In [ ]:
df['drug'].nunique()

In [ ]:
prescriptions_df[prescriptions_df.subject_id==14878942]

# Approach 5

In [ ]:
patient_data.head()

***We know that there are multiple diagnosis present for one hadm_id. Thus there are no distinguised drug present for the respective diagnosis***

***APPROACH***
- Group all the Diagnosis together and get the unique drugs present in for the diagnosis.
- Get the data like, Dia1, present with how many Diagnosis

In [ ]:
## Unique Drugs present per Diagnosis 
patient_data.groupby(['diagnosis'])['drug'].nunique()

In [ ]:

diag_corr_df = patient_data[['subject_id', 'diagnosis']]

# Perform one-hot encoding for diagnoses
diagnoses_encoded = pd.get_dummies(diag_corr_df['diagnosis'])

# Calculate correlation matrix
correlation_matrix = diagnoses_encoded.corr()

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap of Diagnoses")
plt.show()

In [ ]:
correlation_matrix

In [ ]:
N= 5
# Find pairs with highest correlation coefficients
high_corr_pairs = correlation_matrix.unstack().sort_values(ascending=False)
# Exclude pairs with correlation 1 (diagnosis perfectly correlated with itself)
high_corr_pairs = high_corr_pairs[high_corr_pairs < 1]
# Print top N pairs
top_pairs = high_corr_pairs.head(N)
print("Top correlated pairs of diagnoses:")
top_pairs


 ### Get the most closely related N diagnosis and pair with each other

In [ ]:
diagnosis_of_interest = 'Congenital Anomalies'
correlation_with_interest = correlation_matrix[diagnosis_of_interest]#.drop(diagnosis_of_interest)
correlation_with_interest
correlation_with_interest.abs().nlargest(5)

In [ ]:
correlation_with_interest = correlation_matrix[diagnosis_of_interest]#.drop(diagnosis_of_interest)
correlation_with_interest
correlation_with_interest.abs().nlargest(5)

In [ ]:
diag_list = patient_data.diagnosis.unique()
diag_similar_list = []
for diagnosis_of_interest in diag_list:
    # Extract correlations with the diagnosis of interest
    correlation_with_interest = correlation_matrix[diagnosis_of_interest]#.drop(diagnosis_of_interest)

    # Find top 3 diagnoses with highest correlation coefficients
    top_related_diagnoses = correlation_with_interest.abs().nlargest(6)
    diag_similar_list.append(top_related_diagnoses.index.tolist())

    print("Top 3 diagnoses closely related to:", diagnosis_of_interest)
    print(top_related_diagnoses)
    print('*'*20*6)


## Co-Occurance Matrix

In [ ]:
patient_data.shape

In [ ]:
patient_data.hadm_id.nunique(), patient_data.subject_id.nunique()

#### Group the hadm_id such that it get the unique diagnosis and the count

***BUILD THE COOUCCRANCE MATRIX***

# Approach 6



- [Load Data fro DB](#Load-Data-from-DB)
- [Start Recommendation Process](#Start-Recommendation-Process)
- [Approach 6](#Approach-6)

In [26]:
# Group diagnoses for each patient into lists
patient_data_grouped = patient_data.groupby(['hadm_id'])['diagnosis'].apply(lambda x: sorted(x)).reset_index()
# Rename columns
patient_data_grouped.columns = ['hadm_id', 'diagnosis']

# Unique Diagnosis per Admissions
patient_data_grouped['diagnosis'] = patient_data_grouped['diagnosis'].apply(lambda x : list(set(x)))

# Print the resulting DataFrame
patient_data_grouped['len'] = patient_data_grouped['diagnosis'].apply(lambda x:len(set(x)))
patient_data_grouped.head()

,hadm_id,diagnosis,len
0,20000094,"[Symptoms, Signs, and Ill-defined Conditions, ...",7
1,20008372,[Diseases of the Nervous System and Sense Orga...,9
2,20012747,[External causes of injury and supplemental cl...,3
3,20014035,[Diseases of the Blood and Blood-forming Organ...,8
4,20022675,[Diseases of the Musculoskeletal System and Co...,6


## Cooccuarance and Corelation for Diagnosis

In [27]:
# Create a list of all unique diagnoses
all_diagnoses = sorted(patient_data['diagnosis'].unique())
# Initialize blank co-occurrence matrix
co_occurrence_matrix = pd.DataFrame(0, index=all_diagnoses, columns=all_diagnoses)

# Iterate over each patient's diagnoses and update co-occurrence matrix
i=0
for diagnoses_list in patient_data_grouped['diagnosis']:
    i+=1
    output = f"Counter: {i}"
    print(output, end='\r')
    
    for diag1, diag2 in combinations(diagnoses_list, 2):
        co_occurrence_matrix.at[diag1, diag2] += 1
        co_occurrence_matrix.at[diag2, diag1] += 1
        
    
co_occurrence_matrix

,"Complications of Pregnancy, Childbirth, and the Puerperium",Congenital Anomalies,Diseases of the Blood and Blood-forming Organs,Diseases of the Circulatory System,Diseases of the Digestive System,Diseases of the Genitourinary System,Diseases of the Musculoskeletal System and Connective Tissue,Diseases of the Nervous System and Sense Organs,Diseases of the Respiratory System,Diseases of the Skin and Subcutaneous Tissue,"Diseases of the eye, adnexa and mastoid process","Endocrine, Nutritional, and Metabolic Diseases",External causes of injury and supplemental classification,Factors influencing health status and contact with health services,Infectious and Parasitic Diseases,Injury and Poisoning,Mental Disorders,Neoplasms,"Symptoms, Signs, and Ill-defined Conditions"
"Complications of Pregnancy, Childbirth, and the Puerperium",0,1,19,3,18,8,4,10,16,1,0,18,86,46,6,2,19,2,13
Congenital Anomalies,1,0,34,67,41,42,30,42,39,12,1,34,60,21,19,22,42,10,45
Diseases of the Blood and Blood-forming Organs,19,34,0,516,370,345,207,276,257,100,21,239,525,281,171,197,301,158,365
Diseases of the Circulatory System,3,67,516,0,683,625,459,561,516,174,46,585,1063,491,276,347,574,270,670
Diseases of the Digestive System,18,41,370,683,0,399,312,373,362,115,31,345,711,337,210,222,461,183,490
Diseases of the Genitourinary System,8,42,345,625,399,0,269,341,305,109,22,303,608,290,211,220,336,157,414
Diseases of the Musculoskeletal System and Connective Tissue,4,30,207,459,312,269,0,281,222,101,23,229,456,216,143,152,276,104,304
Diseases of the Nervous System and Sense Organs,10,42,276,561,373,341,281,0,305,96,24,309,565,251,178,216,379,140,401
Diseases of the Respiratory System,16,39,257,516,362,305,222,305,0,80,18,261,480,235,155,171,320,131,344
Diseases of the Skin and Subcutaneous Tissue,1,12,100,174,115,109,101,96,80,0,7,101,175,68,77,83,89,46,100


***FILTER THE TOP N DIAGNOSIS RELATED TO THE TARGET DIAGNOSIS***

In [28]:
# Initialize a dictionary to store top N diagnoses for each diagnosis
top_N_diagnoses = {}
N =4
# Iterate over each diagnosis
for diagnosis in co_occurrence_matrix.index:
    # Sort co-occurrence values for the current diagnosis
    sorted_diagnoses = co_occurrence_matrix.loc[diagnosis].sort_values(ascending=False)
    # Exclude the current diagnosis itself
    sorted_diagnoses = sorted_diagnoses.drop(diagnosis)
    # Select top N diagnoses
    top_N = sorted_diagnoses.head(N).index.tolist()
    # Store top N diagnoses in the dictionary
    top_N_diagnoses[diagnosis] = top_N

# Print top N diagnoses for each diagnosis
for diagnosis, top_N in top_N_diagnoses.items():
    print(diagnosis)
    for i, v in enumerate(top_N):
        print(i+1, ': ', v)
    print('*'*100)


Complications of Pregnancy, Childbirth, and the Puerperium
1 :  External causes of injury and supplemental classification
2 :  Factors influencing health status and contact with health services
3 :  Diseases of the Blood and Blood-forming Organs
4 :  Mental Disorders
****************************************************************************************************
Congenital Anomalies
1 :  Diseases of the Circulatory System
2 :  External causes of injury and supplemental classification
3 :  Symptoms, Signs, and Ill-defined Conditions
4 :  Diseases of the Genitourinary System
****************************************************************************************************
Diseases of the Blood and Blood-forming Organs
1 :  External causes of injury and supplemental classification
2 :  Diseases of the Circulatory System
3 :  Diseases of the Digestive System
4 :  Symptoms, Signs, and Ill-defined Conditions
******************************************************************************

## Convert the Co-Occuarance Matrix to Correlation Matrix

In [29]:
# Calculate the correlation matrix
correlation_matrix = co_occurrence_matrix.corr()

# Print the correlation matrix
correlation_matrix

,"Complications of Pregnancy, Childbirth, and the Puerperium",Congenital Anomalies,Diseases of the Blood and Blood-forming Organs,Diseases of the Circulatory System,Diseases of the Digestive System,Diseases of the Genitourinary System,Diseases of the Musculoskeletal System and Connective Tissue,Diseases of the Nervous System and Sense Organs,Diseases of the Respiratory System,Diseases of the Skin and Subcutaneous Tissue,"Diseases of the eye, adnexa and mastoid process","Endocrine, Nutritional, and Metabolic Diseases",External causes of injury and supplemental classification,Factors influencing health status and contact with health services,Infectious and Parasitic Diseases,Injury and Poisoning,Mental Disorders,Neoplasms,"Symptoms, Signs, and Ill-defined Conditions"
"Complications of Pregnancy, Childbirth, and the Puerperium",1.000000,0.467060,0.530056,0.738647,0.547554,0.569018,0.574047,0.544135,0.495403,0.521563,0.586310,0.361321,-0.129015,0.396708,0.504078,0.697501,0.533857,0.618512,0.563131
Congenital Anomalies,0.467060,1.000000,0.874714,0.624583,0.854533,0.847792,0.914157,0.856415,0.873799,0.940329,0.783363,0.894005,0.680141,0.869299,0.945319,0.908255,0.846680,0.947916,0.825013
Diseases of the Blood and Blood-forming Organs,0.530056,0.874714,1.000000,0.580130,0.746977,0.758831,0.877980,0.825537,0.835093,0.846831,0.809032,0.797212,0.557911,0.755574,0.845898,0.848554,0.808732,0.861913,0.752397
Diseases of the Circulatory System,0.738647,0.624583,0.580130,1.000000,0.518587,0.515384,0.591539,0.570222,0.564241,0.630159,0.507221,0.453446,0.175128,0.504198,0.671203,0.670541,0.590767,0.649600,0.524978
Diseases of the Digestive System,0.547554,0.854533,0.746977,0.518587,1.000000,0.756713,0.785448,0.773488,0.753300,0.818323,0.720047,0.734115,0.466066,0.715471,0.800008,0.841914,0.683216,0.827954,0.682742
Diseases of the Genitourinary System,0.569018,0.847792,0.758831,0.515384,0.756713,1.000000,0.824089,0.781681,0.803332,0.847294,0.783812,0.769537,0.529088,0.752956,0.790418,0.834214,0.802009,0.870434,0.734014
Diseases of the Musculoskeletal System and Connective Tissue,0.574047,0.914157,0.877980,0.591539,0.785448,0.824089,1.000000,0.802089,0.863523,0.853136,0.779138,0.815346,0.583491,0.792911,0.878937,0.893308,0.817185,0.921060,0.793745
Diseases of the Nervous System and Sense Organs,0.544135,0.856415,0.825537,0.570222,0.773488,0.781681,0.802089,1.000000,0.794516,0.867878,0.746464,0.759535,0.552077,0.772801,0.844192,0.824455,0.737327,0.875518,0.731911
Diseases of the Respiratory System,0.495403,0.873799,0.835093,0.564241,0.753300,0.803332,0.863523,0.794516,1.000000,0.894674,0.803590,0.795279,0.615639,0.785061,0.871465,0.867047,0.785991,0.888176,0.775146
Diseases of the Skin and Subcutaneous Tissue,0.521563,0.940329,0.846831,0.630159,0.818323,0.847294,0.853136,0.867878,0.894674,1.000000,0.771626,0.840293,0.610230,0.834609,0.882132,0.878676,0.878560,0.924381,0.859485


In [30]:
# Assuming correlation_matrix is your correlation matrix DataFrame
N =4

def top_n_features(correlation_matrix, feature, n):
    # Exclude the feature itself
    correlations = correlation_matrix[feature]#.drop(feature)
    # Sort correlations by absolute values in descending order
    sorted_correlations = correlations.abs().sort_values(ascending=False)
    # Get the top N features
    top_n = sorted_correlations.head(n)
    return top_n

similar_diagnosis_list = []
similar_diagnosis_dict = {}
# Example usage: Find top 5 features correlated with 'diag1'
for diagnosis in co_occurrence_matrix.index:
    top_features = top_n_features(correlation_matrix, diagnosis, N)
    similar_diagnosis_list.append(top_features.index.tolist())
    ## Fame th dict with diagnosis and related diagnosis
    key_diagnosis = top_features.index.tolist()
    similar_diagnosis_dict[key_diagnosis[0]] = key_diagnosis[1:]
    print(top_features)
    print('*'*100)

Complications of Pregnancy, Childbirth, and the Puerperium    1.000000
Diseases of the Circulatory System                            0.738647
Injury and Poisoning                                          0.697501
Neoplasms                                                     0.618512
Name: Complications of Pregnancy, Childbirth, and the Puerperium, dtype: float64
****************************************************************************************************
Congenital Anomalies                            1.000000
Neoplasms                                       0.947916
Infectious and Parasitic Diseases               0.945319
Diseases of the Skin and Subcutaneous Tissue    0.940329
Name: Congenital Anomalies, dtype: float64
****************************************************************************************************
Diseases of the Blood and Blood-forming Organs                  1.000000
Diseases of the Musculoskeletal System and Connective Tissue    0.877980
Congenital Anoma

## Filter the Drugs that contains the above combination of Diagnosis

### ***Find the Co - Occuarance Matrix for the Drug and Diagnosis***

# This block of code is an alternative for getting the recommendation for diagnosis and disease

In [92]:
len([1.        , 0.42318166, 0.44209958, 0.45787184, 0.4757196 ,
        0.44467012, 0.49634168, 0.47523164, 0.420814  , 0.41019136,
        0.45795181, 0.45860574, 0.50086362, 0.49221405, 0.39463317,
        0.44331175, 0.49039956, 0.46120869, 0.47000874])

19

In [89]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Create a User-Item Matrix
# Assume you have a dataset where rows represent diagnoses, columns represent drugs,
# and the values represent the frequency of drug prescriptions for each diagnosis.
# # Replace this with your actual dataset.
# user_item_matrix = np.array([
#     [0, 1, 1, 0, 0],
#     [1, 0, 0, 1, 0],
#     [0, 1, 1, 1, 0],
#     [0, 0, 1, 1, 1],
# ])
# Step 1: Calculate the co-occurrence matrix
user_item_matrix = pd.crosstab(patient_data['diagnosis'], patient_data['drug'])

# Step 2: Calculate Similarity Scores
# Calculate cosine similarity between diagnoses
similarity_matrix = cosine_similarity(user_item_matrix)

# Step 3: Recommendation Generation
def recommend_drugs(diagnosis, user_item_matrix, similarity_matrix, top_n=5):
    # Find the index of the given diagnosis
    diagnosis_index = np.where(diagnosis_list == diagnosis)[0][0]
    # Get similarity scores for all diagnoses
    similarity_scores = similarity_matrix[diagnosis_index]
    # Sort diagnoses by similarity scores in descending order
    similar_diagnoses_indices = np.argsort(similarity_scores)[::-1]
    # Exclude the given diagnosis itself
    similar_diagnoses_indices = similar_diagnoses_indices[similar_diagnoses_indices != diagnosis_index]
    # Get the top N similar diagnoses
    top_similar_diagnoses_indices = similar_diagnoses_indices[:top_n]
    # Get drugs prescribed for top similar diagnoses
    diag_indices = user_item_matrix.index.tolist()
    recommended_drugs = [diag_indices[i] for i in top_similar_diagnoses_indices]
#     recommended_drugs = np.sum(user_item_matrix[top_similar_diagnoses_indices], axis=0)
    # Exclude drugs already prescribed for the given diagnosis
#     recommended_drugs_indices = np.where(user_item_matrix[diagnosis_index] == 0)[0]
#     recommended_drugs = recommended_drugs[recommended_drugs_indices]
#     # Get top N recommended drugs
#     top_recommended_drugs_indices = np.argsort(recommended_drugs)[::-1][:top_n]
#     return [drugs[i] for i in top_recommended_drugs_indices]
    return recommended_drugs


# Example usage:

diagnosis_list = patient_data['diagnosis'].unique()
drugs = patient_data['drug'].unique()
diagnosis = 'Congenital Anomalies'
top_n = 3
recommended_drugs = recommend_drugs(diagnosis, user_item_matrix, similarity_matrix, top_n)
print(f"Recommended drugs for '{diagnosis}': {recommended_drugs}")


Recommended drugs for 'Congenital Anomalies': ['Symptoms, Signs, and Ill-defined Conditions', 'Diseases of the Digestive System', 'External causes of injury and supplemental classification']


### Another Way of CO occuarance Matrix and correlation

In [31]:
# Step 1: Calculate the co-occurrence matrix
co_occur_drug_diagn_matrix = pd.crosstab(patient_data['diagnosis'], patient_data['drug'])

# Step 2: Normalize the co-occurrence matrix
co_occur_drug_diagn_matrix_normalize = normalize(co_occur_drug_diagn_matrix, axis=1)

# Convert the normalized matrix back to a DataFrame
co_occur_drug_diagn_matrix_normalize_df = pd.DataFrame(co_occur_drug_diagn_matrix_normalize, 
                                                  index=co_occur_drug_diagn_matrix.index, columns=co_occur_drug_diagn_matrix.columns)

# Step 3: Find the top correlated drugs for each diagnosis
top_correlated_drugs = {}
for diagnosis in co_occur_drug_diagn_matrix_normalize_df.index:
    correlated_drugs = co_occur_drug_diagn_matrix_normalize_df.loc[diagnosis].sort_values(ascending=False).head(10)
    top_correlated_drugs[diagnosis] = correlated_drugs.index.tolist()

# Convert the dictionary to a DataFrame
top_correlated_drugs_df = pd.DataFrame(top_correlated_drugs)

(top_correlated_drugs_df)

,"Complications of Pregnancy, Childbirth, and the Puerperium",Congenital Anomalies,Diseases of the Blood and Blood-forming Organs,Diseases of the Circulatory System,Diseases of the Digestive System,Diseases of the Genitourinary System,Diseases of the Musculoskeletal System and Connective Tissue,Diseases of the Nervous System and Sense Organs,Diseases of the Respiratory System,Diseases of the Skin and Subcutaneous Tissue,"Diseases of the eye, adnexa and mastoid process","Endocrine, Nutritional, and Metabolic Diseases",External causes of injury and supplemental classification,Factors influencing health status and contact with health services,Infectious and Parasitic Diseases,Injury and Poisoning,Mental Disorders,Neoplasms,"Symptoms, Signs, and Ill-defined Conditions"
0,Acetaminophen,0.9% Sodium Chloride,Insulin,Insulin,0.9% Sodium Chloride,Insulin,0.9% Sodium Chloride,Insulin,0.9% Sodium Chloride,Insulin,Potassium Chloride,0.9% Sodium Chloride,0.9% Sodium Chloride,Insulin,0.9% Sodium Chloride,Insulin,Insulin,0.9% Sodium Chloride,0.9% Sodium Chloride
1,Docusate Sodium,Sodium Chloride 0.9% Flush,0.9% Sodium Chloride,0.9% Sodium Chloride,Potassium Chloride,0.9% Sodium Chloride,Sodium Chloride 0.9% Flush,0.9% Sodium Chloride,Insulin,0.9% Sodium Chloride,Insulin,Insulin,Insulin,Sodium Chloride 0.9% Flush,Insulin,0.9% Sodium Chloride,0.9% Sodium Chloride,Potassium Chloride,Insulin
2,Calcium Carbonate,Furosemide,Potassium Chloride,Potassium Chloride,Insulin,Furosemide,Potassium Chloride,Potassium Chloride,Potassium Chloride,Iso-Osmotic Dextrose,Sodium Chloride 0.9% Flush,Potassium Chloride,Potassium Chloride,Potassium Chloride,Potassium Chloride,Potassium Chloride,Potassium Chloride,Furosemide,Potassium Chloride
3,Ibuprofen,Potassium Chloride,Furosemide,Sodium Chloride 0.9% Flush,Sodium Chloride 0.9% Flush,Potassium Chloride,Insulin,Sodium Chloride 0.9% Flush,Furosemide,Potassium Chloride,Acetaminophen,Sodium Chloride 0.9% Flush,Sodium Chloride 0.9% Flush,Furosemide,Furosemide,Sodium Chloride 0.9% Flush,Sodium Chloride 0.9% Flush,Sodium Chloride 0.9% Flush,Sodium Chloride 0.9% Flush
4,Milk of Magnesia,5% Dextrose,Sodium Chloride 0.9% Flush,Furosemide,Furosemide,Sodium Chloride 0.9% Flush,Acetaminophen,Acetaminophen,Sodium Chloride 0.9% Flush,Sodium Chloride 0.9% Flush,Bag,Furosemide,Acetaminophen,Acetaminophen,Sodium Chloride 0.9% Flush,Furosemide,Acetaminophen,Insulin,Furosemide
5,Sodium Chloride 0.9% Flush,Magnesium Sulfate,Acetaminophen,Acetaminophen,Acetaminophen,Acetaminophen,Furosemide,Furosemide,5% Dextrose,Vancomycin,Sodium Chloride 0.9%,5% Dextrose,Furosemide,0.9% Sodium Chloride,5% Dextrose,Acetaminophen,Furosemide,Acetaminophen,Acetaminophen
6,Bisacodyl,Acetaminophen,5% Dextrose,5% Dextrose,5% Dextrose,5% Dextrose,Heparin,5% Dextrose,Acetaminophen,Furosemide,0.9% Sodium Chloride,Acetaminophen,5% Dextrose,Bag,Iso-Osmotic Dextrose,5% Dextrose,Heparin,5% Dextrose,5% Dextrose
7,Simethicone,Metoprolol Tartrate,Magnesium Sulfate,Metoprolol Tartrate,Magnesium Sulfate,Metoprolol Tartrate,Magnesium Sulfate,Heparin,Metoprolol Tartrate,Acetaminophen,Furosemide,Metoprolol Tartrate,Magnesium Sulfate,Sodium Chloride 0.9%,Magnesium Sulfate,Iso-Osmotic Dextrose,Magnesium Sulfate,Magnesium Sulfate,Heparin
8,Lactated Ringers,Iso-Osmotic Dextrose,Metoprolol Tartrate,Heparin,Heparin,Magnesium Sulfate,5% Dextrose,Magnesium Sulfate,Magnesium Sulfate,5% Dextrose,Heparin,Heparin,Heparin,Heparin,Acetaminophen,Magnesium Sulfate,5% Dextrose,Bag,Magnesium Sulfate
9,Ketorolac,Insulin,Bag,Magnesium Sulfate,Bag,Iso-Osmotic Dextrose,Docusate Sodium,Iso-Osmotic Dextrose,Iso-Osmotic Dextrose,Magnesium Sulfate,Magnesium Sulfate,Magnesium Sulfate,Metoprolol Tartrate,Magnesium Sulfate,Bag,Bag,Bag,Heparin,Bag


## Structure the data in the following format

    {diag1: [(all the related diagnosis), (all the related drug)],
    diag2: [(all the related diagnosis), (all the related drug)],
    .....    }

In [32]:
structured_data = {}

for diagnosis, related_diagnoses in similar_diagnosis_dict.items():
    related_drugs = top_correlated_drugs[diagnosis]
    structured_data[diagnosis] = [tuple(related_diagnoses), tuple(related_drugs)]

structured_data


{'Complications of Pregnancy, Childbirth, and the Puerperium': [('Diseases of the Circulatory System',
   'Injury and Poisoning',
   'Neoplasms'),
  ('Acetaminophen',
   'Docusate Sodium',
   'Calcium Carbonate',
   'Ibuprofen',
   'Milk of Magnesia',
   'Sodium Chloride 0.9%  Flush',
   'Bisacodyl',
   'Simethicone',
   'Lactated Ringers',
   'Ketorolac')],
 'Congenital Anomalies': [('Neoplasms',
   'Infectious and Parasitic Diseases',
   'Diseases of the Skin and Subcutaneous Tissue'),
  ('0.9% Sodium Chloride',
   'Sodium Chloride 0.9%  Flush',
   'Furosemide',
   'Potassium Chloride',
   '5% Dextrose',
   'Magnesium Sulfate',
   'Acetaminophen',
   'Metoprolol Tartrate',
   'Iso-Osmotic Dextrose',
   'Insulin')],
 'Diseases of the Blood and Blood-forming Organs': [('Diseases of the Musculoskeletal System and Connective Tissue',
   'Congenital Anomalies',
   'Neoplasms'),
  ('Insulin',
   '0.9% Sodium Chloride',
   'Potassium Chloride',
   'Furosemide',
   'Sodium Chloride 0.9%  Flu

In [ ]:
'Congenital Anomalies': ['Symptoms, Signs, and Ill-defined Conditions', 'Diseases of the Digestive System', 'External causes of injury and supplemental classification', 'Diseases of the Musculoskeletal System and Connective Tissue', 'Mental Disorders']

In [45]:
import json
with open('..\Data\structured_data.json', 'w') as json_file:
    json.dump(structured_data, json_file)

## TEST


## ***what % of a drug is prescribed to a patient with respect to all the patients***

In [33]:
filtered_df = patient_data[patient_data.diagnosis=='Diseases of the Nervous System and Sense Organs'].groupby(['hadm_id'])['drug'].value_counts().reset_index()
filtered_df.head()

,hadm_id,drug,count
0,20008372,Insulin,42
1,20008372,Alteplase (Catheter Clearance),8
2,20008372,Labetalol,8
3,20008372,Sodium Chloride 0.9%,8
4,20008372,Syringe (SW),8


In [34]:
# Calculate the count of unique patients for each drug
patients_per_drug = filtered_df.groupby('drug')['hadm_id'].nunique()

# Calculate the total number of unique patients
total_patients = filtered_df['hadm_id'].nunique()

# Calculate the percentage of patients for each drug
percentage_per_drug = (patients_per_drug / total_patients) * 100

top_drugs = percentage_per_drug.nlargest(10)

# Print the percentage of patients for each drug
(top_drugs)


drug
Sodium Chloride 0.9%  Flush    92.987377
Acetaminophen                  74.053296
Heparin                        66.199158
Docusate Sodium                59.046283
Senna                          58.485273
0.9% Sodium Chloride           46.844320
Ondansetron                    44.740533
Insulin                        36.605891
Dextrose 50%                   35.764376
Potassium Chloride             35.624123
Name: hadm_id, dtype: float64

In [40]:
patient_data.diagnosis.value_counts(normalize=True)*100

diagnosis
External causes of injury and supplemental classification             11.203401
Diseases of the Circulatory System                                    10.766390
Diseases of the Digestive System                                       7.583684
Symptoms, Signs, and Ill-defined Conditions                            7.312184
Diseases of the Genitourinary System                                   6.800218
Mental Disorders                                                       6.531092
Diseases of the Nervous System and Sense Organs                        6.510742
Diseases of the Respiratory System                                     6.371007
Diseases of the Blood and Blood-forming Organs                         6.237377
Endocrine, Nutritional, and Metabolic Diseases                         5.509194
Factors influencing health status and contact with health services     5.149851
Injury and Poisoning                                                   4.980608
Infectious and Parasitic Disea

In [35]:
## Get all the Drug list for a diagnosis

open_list_of_drugs = []
for i,v in structured_data.items():
    open_list_of_drugs.append((i,list(v[1])))


# Initialize a list to store dictionaries representing each row
data = []

for diag, provided_drugs in open_list_of_drugs:
    # Filter top drugs DataFrame for drugs present in the provided list
    matching_drugs = top_drugs[top_drugs.index.isin(provided_drugs)]

    # Create a dictionary representing the current row
    row_data = {
        'Diagnosis': diag,
        'Correct': len(matching_drugs.index.tolist()),
#         'Drugs': ', '.join(matching_drugs.index.tolist())
    }

    # Append the dictionary to the list
    data.append(row_data)

# Create a DataFrame from the list of dictionaries
result_df = pd.DataFrame(data)

(result_df.sort_values(['Correct'],
                       ascending=False))


,Diagnosis,Correct
6,Diseases of the Musculoskeletal System and Con...,7
18,"Symptoms, Signs, and Ill-defined Conditions",6
13,Factors influencing health status and contact ...,6
17,Neoplasms,6
3,Diseases of the Circulatory System,6
4,Diseases of the Digestive System,6
16,Mental Disorders,6
7,Diseases of the Nervous System and Sense Organs,6
10,"Diseases of the eye, adnexa and mastoid process",6
11,"Endocrine, Nutritional, and Metabolic Diseases",6


In [36]:
patient_data.groupby('diagnosis')['drug'].nunique()

diagnosis
Complications of Pregnancy, Childbirth, and the Puerperium             145
Congenital Anomalies                                                   433
Diseases of the Blood and Blood-forming Organs                         897
Diseases of the Circulatory System                                    1017
Diseases of the Digestive System                                       919
Diseases of the Genitourinary System                                   882
Diseases of the Musculoskeletal System and Connective Tissue           787
Diseases of the Nervous System and Sense Organs                        904
Diseases of the Respiratory System                                     879
Diseases of the Skin and Subcutaneous Tissue                           587
Diseases of the eye, adnexa and mastoid process                        284
Endocrine, Nutritional, and Metabolic Diseases                         810
External causes of injury and supplemental classification             1046
Factors influen

In [37]:
# Calculate the total correct count
total_correct_count = sum(result_df['Correct'].values)

# Calculate the total count of drugs
total_drug_count = 10 * len(result_df['Correct'])

# Calculate the overall accuracy
overall_accuracy = total_correct_count / total_drug_count

print("Overall Accuracy:", overall_accuracy)


Overall Accuracy: 0.5526315789473685


In [38]:
total_drug_count

190